# Loading GeoJSON files into SpatiaLite


This notebook demonstrates how to load geojson boundary files into a SpatiaLite database based on a [recipe](https://datasette.readthedocs.io/en/stable/spatialite.html#importing-geojson-polygons-using-shapely) cribbed from Simon Willison.

A wide selection of UK based geojson boundary files can be found here: https://github.com/martinjc/UK-GeoJSON/

In [1]:
!rm lsoas.db

In [13]:
import sqlite3
conn = sqlite3.connect('lsoas.db')

# Enable SpatialLite extension
conn.enable_load_extension(True)

# !whereis mod_spatialite
conn.load_extension('/usr/lib/x86_64-linux-gnu/mod_spatialite.so')

# Initialise spatial table support
conn.execute('SELECT InitSpatialMetadata(1)')

#Create a table for lsoas
conn.execute('DROP TABLE IF EXISTS lsoas;')
conn.execute('CREATE TABLE lsoas (id integer primary key, name text);')

# Add a MULTIPOLYGON Geometry column
conn.execute("SELECT AddGeometryColumn('lsoas', 'geom', 4326, 'MULTIPOLYGON', 2);")
# Add a spatial index against the new column
conn.execute("SELECT CreateSpatialIndex('lsoas', 'geom');")

## Loading GeoJSON into SpatiaLite

In [8]:
import requests
import json

#Need to provide a lookup for this?
la_code = 'E06000046'

url = 'https://raw.githubusercontent.com/martinjc/UK-GeoJSON/master/json/statistical/eng/lsoa_by_lad/{}.json'.format(la_code)
data = requests.get(url)


import os
geodir = 'geodata'
if not os.path.exists(geodir):
    os.makedirs(geodir)
with open('{}/{}.geojson'.format(geodir,la_code), 'w') as f:
        f.write(data.text)
  


I haven't found an effective way of loading geojson into `spatialite` directly, or via `geopandas`. But importing data into `spatilite` via a shapefile seems to work, so we can create a workaround:

- load geojson into `geopandas`;
- save as shapefile;
- load shapefile into `spatialite`


In [11]:
import geopandas as gpd

#This may kill MyBinder depending on Binder resource allocation?
fn = "geodata/E06000046.geojson"

#We can also load in straight from url
gdf = gpd.read_file(fn)
gdf.head()

,LSOA11CD,LSOA11NM,geometry
0,E01017282,Isle of Wight 006A,"POLYGON ((-1.17165606546966 50.72114919198948,..."
1,E01017283,Isle of Wight 006B,POLYGON ((-1.168065429754894 50.72135794384606...
2,E01017284,Isle of Wight 010A,"POLYGON ((-1.08034222501772 50.69064111644322,..."
3,E01017285,Isle of Wight 010B,POLYGON ((-1.069832758525084 50.68675112435623...
4,E01017286,Isle of Wight 005A,"POLYGON ((-1.20066878623609 50.7341352724568, ..."


In [22]:
#Save as a shapefile
#I think the driver attribute can be assumed?
gdf.to_file(driver = 'ESRI Shapefile', filename= "result.shp", encoding='utf8')

In [52]:
#4326 has the lat / long flipped cf the wgs84 projection?

! spatialite lsoas.db ".loadshp ./result adminboundaries2 UTF8 4326"

SpatiaLite version ..: 4.3.0a	Supported Extensions:
	- 'VirtualShape'	[direct Shapefile access]
	- 'VirtualDbf'		[direct DBF access]
	- 'VirtualXL'		[direct XLS access]
	- 'VirtualText'		[direct CSV/TXT access]
	- 'VirtualNetwork'	[Dijkstra shortest path]
	- 'RTree'		[Spatial Index - R*Tree]
	- 'MbrCache'		[Spatial Index - MBR cache]
	- 'VirtualSpatialIndex'	[R*Tree metahandler]
	- 'VirtualElementary'	[ElemGeoms metahandler]
	- 'VirtualXPath'	[XML Path Language - XPath]
	- 'VirtualFDO'		[FDO-OGR interoperability]
	- 'VirtualGPKG'	[OGC GeoPackage interoperability]
	- 'VirtualBBox'		[BoundingBox tables]
	- 'SpatiaLite'		[Spatial SQL - OGC]
PROJ.4 version ......: Rel. 4.9.3, 15 August 2016
GEOS version ........: 3.6.2-CAPI-1.10.2 4d2925d6
TARGET CPU ..........: x86_64-linux-gnu
Loading shapefile at './result' into SQLite table 'adminboundaries2'

BEGIN;
CREATE TABLE "adminboundaries2" (
"PK_UID" INTEGER PRIMARY KEY AUTOINCREMENT,
"LSOA11CD" TEXT,
"LSOA11NM" TEXT);
SELECT AddGeometryColumn

In [ ]:
#lon / lat vs lat / lon 
#https://github.com/Leaflet/Leaflet/issues/1455#issuecomment-385273232

In [5]:
import requests, json
#import pandas as pd
import geopandas as gpd
from shapely.geometry import shape 


def query_datasette_api(q,db_url = "http://localhost:8001/adminboundaries.json",
                        dataframe=False, geojsoncol=None):
    ''' Simple query to datasette endpoint. Return response as a dict or pandas dataframe. '''
    params = {'sql': q}

    r = requests.get(db_url, params=params)
    jdata=r.json()
    if dataframe:
        df=gpd.GeoDataFrame(jdata['rows'])
        df.columns = jdata['columns']
        for c in df.columns:
            #Need a better way to identify geo columns?
            #Also, this should really be a geopandas dataframe, with the json column as a geometry column
            if c.startswith('AsGeoJSON'):
                df[c]=df[c].apply(json.loads)
                cn=c.replace('AsGeoJSON(','').replace(')','')
                df[cn]=df[c].apply(shape)
        return df
    
    #Need a better way to identify geo columns?
    ix = [jdata['columns'].index(c) for c in jdata['columns'] if c.startswith('AsGeoJSON')]
    for i in ix:
        for i2 in range(len(jdata['rows'])):
            jdata['rows'][i2][i]=json.loads(jdata['rows'][i2][i])

    geojsoncol  = geojsoncol if geojsoncol and geojsoncol.startswith('AsGeoJSON') else 'AsGeoJSON({})'.format(geojsoncol)
    if geojsoncol and geojsoncol in jdata['columns']:
        return jdata['rows'][jdata['columns'].index(geojsoncol)][0]
    return jdata

In [1]:
#Datasette currently requires specific version of pip?
#!pip install --upgrade click==6.7

# Example of running datasette server from inside a code cell
# via https://nbviewer.jupyter.org/gist/minrk/622080cf8af787734805d12bec4ae302from threading import Thread
from threading import Thread

def app_in_thread():
    ! datasette lsoas.db --port 8002 --load-extension=/usr/lib/x86_64-linux-gnu/mod_spatialite.so --config sql_time_limit_ms:10000 --cors

t = Thread(target=app_in_thread)
t.start()

Serve! files=('lsoas.db',) on port 8002
[2019-02-28 15:01:19 +0000] [587] [INFO] Goin' Fast @ http://127.0.0.1:8002
[2019-02-28 15:01:19 +0000] [587] [INFO] Starting worker [587]


In [12]:
tmp = query_datasette_api("SELECT LSOA11CD, LSOA11NM, AsGeoJSON(Geometry) FROM adminboundaries2 LIMIT 3",
                    "http://localhost:8002/lsoas.json")
#tmp

[2019-02-28 15:05:58 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8002/lsoas.json?sql=SELECT+LSOA11CD%2C+LSOA11NM%2C+AsGeoJSON%28Geometry%29+FROM+adminboundaries2+LIMIT+3  302 0
[2019-02-28 15:05:58 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8002/lsoas-9218c86.json?sql=SELECT+LSOA11CD%2C+LSOA11NM%2C+AsGeoJSON%28Geometry%29+FROM+adminboundaries2+LIMIT+3  200 11656


In [13]:
import folium
import html
from folium.plugins import MarkerCluster


m=folium.Map( location=[ 50.72114919198948,-1.17165606546966,], zoom_start=12)

#Add boundaries to the map -  need to tidy query to make this more obvious
for boundary in tmp['rows']:
    folium.GeoJson(boundary[2], name='geojson').add_to(m)
                      
m